### Prepare the Silver table as Staging

In [0]:
use catalog uc_prod

In [0]:
-- CREATE TABLE uc_prod.silver.daily_pricing_silver (
--     date_of_pricing DATE,
--     row_id INT,
--     state_name STRING,
--     market_name STRING,
--     productgroup_name STRING,
--     product_name STRING,
--     variety STRING,
--     origin STRING,
--     arrival_in_tonnes DECIMAL(20,2),
--     minimum_price DECIMAL(20,2),
--     maximum_price DECIMAL(20,2),
--     modal_price DECIMAL(20,2),
--     file_arrival_date TIMESTAMP,
--     record_inserted_date TIMESTAMP,
--     record_updated_date TIMESTAMP
-- );


In [0]:
truncate table uc_prod.silver.daily_pricing_silver;
-- delete from uc_prod.watermark_tbl.process_logs where Process_name = 'staging_table_insert' and process_status = 'completed'

In [0]:
-- select * from uc_prod.bronze.daily_pricing_tbl limit 10

In [0]:
insert into uc_prod.silver.daily_pricing_silver
select 
to_date(date_of_pricing, 'dd/MM/yyyy') AS date_of_pricing,
cast(ROW_ID as int),
STATE_NAME,
MARKET_NAME,
PRODUCTGROUP_NAME,
PRODUCT_NAME,
VARIETY,
ORIGIN,
cast(ARRIVAL_IN_TONNES as decimal(20,2)),
cast(MINIMUM_PRICE as decimal(20,2)),
cast(MAXIMUM_PRICE as decimal(20,2)),
cast(MODAL_PRICE as decimal(20,2)),
file_arrival_date,
current_timestamp() as record_inserted_date,
current_timestamp() as record_updated_date
from 
uc_prod.bronze.daily_pricing_tbl
where file_arrival_date > (
  select nvl(max(process_file_date),'1900-01-01') from uc_prod.watermark_tbl.process_logs
  where Process_name = 'staging_table_insert' and process_status = 'completed'
)

In [0]:
-- delete from uc_prod.watermark_tbl.process_logs where Process_name = 'staging_table_insert' and process_status = 'completed'

In [0]:
insert into uc_prod.watermark_tbl.process_logs
(Process_name, Process_file_date,process_status)
select 'staging_table_insert',current_timestamp(),'completed'
from uc_prod.bronze.daily_pricing_tbl


### Testing the duplicate entry